In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from zipfile import ZipFile
from kaggle.api.kaggle_api_extended import KaggleApi
from functools import reduce
# import pandas_datareader as pdr
import matplotlib.pyplot as plt
import datetime
# import altair as alt
# import pymc3 as pm
# import arviz as az
import seaborn as sns

from ast import literal_eval



# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split, TimeSeriesSplit
# from sklearn.preprocessing import MinMaxScaler
# from keras.models import Sequential, load_model
# from keras.layers import *
# from tensorflow.keras.losses import MeanSquaredError
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping
# from sklearn.metrics import mean_squared_error, mean_absolute_error

plt.style.use('fivethirtyeight')
%matplotlib inline   
%config InlineBackend.figure_format = 'retina'

# Import Data

In [2]:
# Kaggle API authentication
api = KaggleApi()
api.authenticate()

In [3]:
# Download stock mkt indicators datasets
"""This dataset has around 64 features which include features extracted 
from OHLC, other index prices such as QQQ(Nasdaq-100 ETF) & S&P 500, 
technical indicators such as Bollinger bands, 
EMA(Exponential Moving Averages), Stochastic %K oscillator, RSI, etc."""
api.dataset_download_files('nikhilkohli/us-stock-market-data-60-extracted-features', 'datasets/stock_mkt_indicators/')

In [4]:
# Unzip datasets file
with ZipFile(
        'datasets/stock_mkt_indicators/us-stock-market-data-60-extracted-features.zip',
        'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall('datasets/stock_mkt_indicators/')

In [5]:
# Import datasets
aapl = pd.read_csv('datasets/stock_mkt_indicators/AAPL.csv')
amzn = pd.read_csv('datasets/stock_mkt_indicators/AMZN.csv')
fb = pd.read_csv('datasets/stock_mkt_indicators/FB.csv')
ge = pd.read_csv('datasets/stock_mkt_indicators/GE.csv')
googl = pd.read_csv('datasets/stock_mkt_indicators/GOOGL.csv')
gs = pd.read_csv('datasets/stock_mkt_indicators/GS.csv')
ibm = pd.read_csv('datasets/stock_mkt_indicators/IBM.csv')
jpm = pd.read_csv('datasets/stock_mkt_indicators/JPM.csv')
msft = pd.read_csv('datasets/stock_mkt_indicators/MSFT.csv')
tsla = pd.read_csv('datasets/stock_mkt_indicators/TSLA.csv')

In [6]:
# Stock name column
aapl['stock'] = "Apple"
amzn['stock'] = "Amazon"
fb['stock'] = "Facebook"
ge['stock'] = "General Electric"
googl['stock'] = "Google"
gs['stock'] = "Goldman Sachs"
ibm['stock'] = "IBM"
jpm['stock'] = "JPMorgan Chase"
msft['stock'] = "Microsoft"
tsla['stock'] = "Tesla"

In [7]:
aapl.columns

Index(['Date', 'Open', 'High', 'Low', 'Close(t)', 'Volume', 'SD20',
       'Upper_Band', 'Lower_Band', 'S_Close(t-1)', 'S_Close(t-2)',
       'S_Close(t-3)', 'S_Close(t-5)', 'S_Open(t-1)', 'MA5', 'MA10', 'MA20',
       'MA50', 'MA200', 'EMA10', 'EMA20', 'EMA50', 'EMA100', 'EMA200', 'MACD',
       'MACD_EMA', 'ATR', 'ADX', 'CCI', 'ROC', 'RSI', 'William%R', 'SO%K',
       'STD5', 'ForceIndex1', 'ForceIndex20', 'Date_col', 'Day', 'DayofWeek',
       'DayofYear', 'Week', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Is_leap_year',
       'Year', 'Month', 'QQQ_Close', 'QQQ(t-1)', 'QQQ(t-2)', 'QQQ(t-5)',
       'QQQ_MA10', 'QQQ_MA20', 'QQQ_MA50', 'SnP_Close', 'SnP(t-1))',
       'SnP(t-5)', 'DJIA_Close', 'DJIA(t-1))', 'DJIA(t-5)', 'Close_forcast',
       'stock'],
      dtype='object')

# Exploratory Data Analysis

In [14]:
# 
aapl_corr_df = pd.DataFrame({'Apple':aapl.corr()['Close(t)']}).sort_values(by = 'Apple', axis = 0, ascending = False)
amzn_corr_df = pd.DataFrame({'Amazon':amzn.corr()['Close(t)']}).sort_values(by = 'Amazon', axis = 0, ascending = False)
fb_corr_df = pd.DataFrame({'Facebook':fb.corr()['Close(t)']}).sort_values(by = 'Facebook', axis = 0, ascending = False)
ge_corr_df = pd.DataFrame({'General Electric':ge.corr()['Close(t)']}).sort_values(by = 'General Electric', axis = 0, ascending = False)
googl_corr_df = pd.DataFrame({'Google':googl.corr()['Close(t)']}).sort_values(by = 'Google', axis = 0, ascending = False)
gs_corr_df = pd.DataFrame({'Goldman Sachs':gs.corr()['Close(t)']}).sort_values(by = 'Goldman Sachs', axis = 0, ascending = False)
ibm_corr_df = pd.DataFrame({'IBM':ibm.corr()['Close(t)']}).sort_values(by = 'IBM', axis = 0, ascending = False)
jpm_corr_df = pd.DataFrame({'JPMorgan Chase':jpm.corr()['Close(t)']}).sort_values(by = 'JPMorgan Chase', axis = 0, ascending = False)
msft_corr_df = pd.DataFrame({'Microsoft':msft.corr()['Close(t)']}).sort_values(by = 'Microsoft', axis = 0, ascending = False)
tsla_corr_df = pd.DataFrame({'Tesla':tsla.corr()['Close(t)']}).sort_values(by = 'Tesla', axis = 0, ascending = False)

In [42]:
# 
aapl_corr_df = aapl_corr_df[aapl_corr_df['Apple'] >= 0.75]
aapl_corr_df['Rank'] = aapl_corr_df['Apple'].rank(axis = 0, method = 'first')
aapl_corr_df.index.name = 'index'
amzn_corr_df = amzn_corr_df[amzn_corr_df['Amazon'] >= 0.75]
amzn_corr_df['Rank'] = amzn_corr_df['Amazon'].rank(axis = 0, method = 'first')
amzn_corr_df.index.name = 'index'
fb_corr_df = fb_corr_df[fb_corr_df['Facebook'] >= 0.75]
fb_corr_df['Rank'] = fb_corr_df['Facebook'].rank(axis = 0, method = 'first')
fb_corr_df.index.name = 'index'
ge_corr_df = ge_corr_df[ge_corr_df['General Electric'] >= 0.75]
ge_corr_df['Rank'] = ge_corr_df['General Electric'].rank(axis = 0, method = 'first')
ge_corr_df.index.name = 'index'
googl_corr_df = googl_corr_df[googl_corr_df['Google'] >= 0.75]
googl_corr_df['Rank'] = googl_corr_df['Google'].rank(axis = 0, method = 'first')
googl_corr_df.index.name = 'index'
gs_corr_df = gs_corr_df[gs_corr_df['Goldman Sachs'] >= 0.75]
gs_corr_df['Rank'] = gs_corr_df['Goldman Sachs'].rank(axis = 0, method = 'first')
gs_corr_df.index.name = 'index'
ibm_corr_df = ibm_corr_df[ibm_corr_df['IBM'] >= 0.75]
ibm_corr_df['Rank'] = ibm_corr_df['IBM'].rank(axis = 0, method = 'first')
ibm_corr_df.index.name = 'index'
jpm_corr_df = jpm_corr_df[jpm_corr_df['JPMorgan Chase'] >= 0.75]
jpm_corr_df['Rank'] = jpm_corr_df['JPMorgan Chase'].rank(axis = 0, method = 'first')
jpm_corr_df.index.name = 'index'
msft_corr_df = msft_corr_df[msft_corr_df['Microsoft'] >= 0.75]
msft_corr_df['Rank'] = msft_corr_df['Microsoft'].rank(axis = 0, method = 'first')
msft_corr_df.index.name = 'index'
tsla_corr_df = tsla_corr_df[tsla_corr_df['Tesla'] >= 0.75]
tsla_corr_df['Rank'] = tsla_corr_df['Tesla'].rank(axis = 0, method = 'first')
tsla_corr_df.index.name = 'index'

In [54]:
# 
df_rank_corr = pd.concat([aapl_corr_df, amzn_corr_df, fb_corr_df, ge_corr_df, googl_corr_df, gs_corr_df,
    ibm_corr_df, jpm_corr_df, msft_corr_df, tsla_corr_df]).groupby(['index']).sum()
df_rank_corr = pd.DataFrame(df_rank_corr['Rank']).sort_values(by = 'Rank', axis = 0, ascending = False).reset_index()
df_rank_corr

,index,Rank
0,Close(t),322.0
1,Low,308.0
2,High,306.0
3,Open,287.0
4,EMA20,281.0
5,EMA50,271.0
6,EMA100,261.0
7,EMA10,251.0
8,EMA200,251.0
9,Close_forcast,229.0
